## Loading data

In [2]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
from sql_db_functions import sql_db_functions as SQLf

In [3]:
conn, cursor = SQLf.connect_sql()

In [4]:
query = f"SELECT product_characteristics.id, Brand_id, Detail,Summary, Brand, Price, Link FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
#rows

In [4]:
conn, cursor = SQLf.connect_sql()

In [5]:
query = f"SELECT Brand_id, image_link, base64 FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows

In [6]:
SQLf.close_connection_db(conn=conn,cursor=cursor)

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(rows ,columns= ['id', 'Brand_id', 'Detail','Summary', 'Brand', 'Price', 'Link'])
#df.head()

,id,Brand_id,Detail,Summary,Brand,Price,Link
0,10,50,"*Type: Top, Shirt \n*Fit: Regular fit \n*Sle...",The top showcased in the images is a stylish s...,HM,15.99,https://www2.hm.com/it_it/productpage.12282720...
1,101,101,"*Type: Top, Shirt \n*Fit: Regular fit \n*Sle...",The top showcased in the images is a fluid fab...,Mango,39.99,https://shop.mango.com/it/it/p/donna/bluse-e-c...
2,4,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,"The shirt from Gucci, as shown in the images, ...",Gucci,1300.00,https://www.gucci.com/it/it/pr/women/ready-to-...
3,2,2,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The showcased shirt is a Gucci Oxford cotton s...,Gucci,650.00,https://www.gucci.com/it/it/pr/women/ready-to-...
4,8,8,*Type: Top \n*Fit: Regular fit \n*Sleeve sty...,The top showcased in the images is a Gucci flo...,Gucci,980.00,https://www.gucci.com/it/it/pr/women/ready-to-...


In [9]:
df_img = pd.DataFrame(rows_img ,columns= ['Brand_id','image_link','base64'])
df_img['image_link'] = df_img['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )
#df_img.head()

,Brand_id,image_link,base64
0,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,1,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,1,https://media.gucci.com/style/White_Center_0_0...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
4,2,https://media.gucci.com/style/DarkGray_Center_...,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [10]:
df['Detail'] = df['Detail'] = df['Detail'].str.replace('\n', ' / ').str.replace(r'\s+', ' ').str.replace('*', ' ')

In [11]:
import numpy as np

In [12]:
final_df = pd.DataFrame()

for parent_row in df.Detail: 
    x = []
    for row in parent_row.split('/'):
        x.append(row.split(':'))

    temp_dataframe = pd.DataFrame(x)
    temp_dataframe[0] = temp_dataframe[0].str.lower()
    temp_dataframe = temp_dataframe.sort_values(by=0)
    temp_dataframe = temp_dataframe.groupby(0)[1].apply(lambda x: ', '.join(x)).reset_index()
        
    temp_dataframe = temp_dataframe.set_index(0).T
    temp_dataframe.columns = temp_dataframe.columns.str.lstrip()
    
    final_df = pd.concat([final_df,temp_dataframe], ignore_index = True)

final_df['seasson'] = final_df['season'].combine_first(final_df['seasson'])
final_df = final_df.drop('season', axis=1)

In [14]:
df = pd.concat([df,final_df], axis=1)

## aaaa